In [2]:
import pandas as pd

In [25]:
def hex_str_to_ascii_string(hex_str):
    new_str = ""
    for letter in [hex_str[i:i + 2] for i in range(0, len(hex_str), 2)]:
        try:
            ascii_val = int(letter,16)
            if ascii_val == 20 or ascii_val >= 30:
                new_str += chr(ascii_val)
            else:
                pass
        except:
            pass
    return new_str.replace(" $","")

def isIn(needles,hay):
    return any([needle in hay for needle in needles])

In [26]:
class Request():
    def __init__(self,request_type):
        self.request_type = request_type
        self.df = pd.DataFrame()
        self.state = "open"
        self.starting_time = None
        self.ending_time = None
        self.duration = None
        self.data_size = None
        self.mtu_size = None
        self.conn_interval = None
        
    def add(self,row):
        
        row_starts_with_Data = hex_str_to_ascii_string(row['Value']).startswith("Data")
        dataframe_starts_with_Data = self.get_data().startswith("Data")
        if(dataframe_starts_with_Data):
            self.df = self.df.append(row)
        else:
            if(row_starts_with_Data):
                self.df = self.df.append(row)
            else:
                if isIn(["READ Rcvd","NOTIFY"],row['Pakettype']):
                    print("Should not add Row")
                else:
                    self.df = self.df.append(row)
    
    def state_pause(self):
        self.state ="pause"
        
    def state_close(self):
        self.state = "close"
        self.starting_time = self.df.iloc[0]['Time.1']
        self.ending_time = self.df.iloc[-1]['Time.1']
        self.duration = self.ending_time - self.starting_time
        
    def state_open(self):
        self.state = "open"
        
    def set_data_size(self,data_size):
        self.data_size = data_size    
        
    def set_mtu_size(self,mtu_size):
        self.mtu_size = mtu_size
        
    def set_conn_interval(self,conn_interval):
        self.conn_interval = conn_interval
    
    def get_data(self,print_conversion=False):
        if "Value" in self.df.columns:
            data = ""
            for i,row in self.df[self.df['Pakettype'].apply(lambda x:isIn(["READ","NOTIFY"],x))].iterrows():
                if not "nan" in row['Value']:
                    data += hex_str_to_ascii_string(row["Value"])
                    if print_conversion:
                        print("org",row["Value"])
                        print("conversion",hex_str_to_ascii_string(row["Value"]))
            return data
        else:
            return ""
    def get_request_numbers(self):
        df_tmp = self.df
        read_requests = len(df_tmp[df_tmp['Opcode'].apply(lambda x:isIn(["Read Request","Read Blob Request"],x))].index)
        read_responses = len(df_tmp[df_tmp['Opcode'].apply(lambda x:isIn(["Read Response","Read Blob Response"],x))].index)
        notifications = len(df_tmp[df_tmp['Opcode'].str.contains("Notification")].index)
        return read_requests,read_responses,notifications
    
    def as_dict(self):
        request_dict = dict()
        request_dict["request_type"] = self.request_type
        request_dict["starting_time"] = self.starting_time 
        request_dict["ending_time"] = self.ending_time 
        request_dict["duration"] = self.duration 
        request_dict["data_size"] = self.data_size 
        request_dict["mtu_size"] = self.mtu_size 
        request_dict["conn_interval"] = self.conn_interval
        request_dict["data"] = self.get_data()
        request_dict["read_requests"],request_dict["read_responses"],request_dict["notifies"] = self.get_request_numbers()
        
        return request_dict    
    
    def __str__(self):
        return str(self.as_dict())

In [27]:
def create_str(length):
    standardOutput = "Data Size " + str(length)
    content = ""
    i = len(standardOutput)
    while i < length:
        content += chr(97+(i%26))
        i+=1
    print(standardOutput+content,len(standardOutput+content))

In [28]:
create_str(64)
'Data Size 64mnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijkl'

Data Size 64mnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijkl 64


'Data Size 64mnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijkl'

In [29]:
check = "446174612053697a65203132386e6f707172737475767778797a6162636465666768696a"
print(len(check))
ascii_str= hex_str_to_ascii_string(check)
len(ascii_str)
ascii_str

72
44
D
61
a
74
t
61
a
20
 
53
S
69
i
7a
z
65
e
20
 
31
1
32
2
38
8
6e
n
6f
o
70
p
71
q
72
r
73
s
74
t
75
u
76
v
77
w
78
x
79
y
7a
z
61
a
62
b
63
c
64
d
65
e
66
f
67
g
68
h
69
i
6a
j


'Data Size 128nopqrstuvwxyzabcdefghij'

In [19]:
hex_str_to_ascii_string("aâ\200\246")

aâ
¦


''